In [16]:
from collections import defaultdict

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from datasets import Dataset
from torch import nn, optim
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
device

device(type='cuda', index=0)

In [2]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

In [3]:
df = pd.read_csv("data/IMDB_Dataset.csv")

In [4]:
sentiment_int = {'positive' : 0, 'negative': 1}
df['sentiment_int'] = df.sentiment.map(sentiment_int)
df.head()

,review,sentiment,sentiment_int
0,One of the other reviewers has mentioned that ...,positive,0
1,A wonderful little production. <br /><br />The...,positive,0
2,I thought this was a wonderful way to spend ti...,positive,0
3,Basically there's a family where a little boy ...,negative,1
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,0


In [5]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=42)

In [56]:
df_train.shape, df_val.shape, df_test.shape

((45000, 3), (2500, 3), (2500, 3))

In [6]:
class ReviewsDataset:

    def __init__(self, reviews, targets, tokenizer, max_len=None):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'review_text': review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

In [7]:
def create_data_loader(df, tokenizer, batch_size, max_len=None):
    ds = ReviewsDataset(
        reviews=df.review.to_numpy(),
        targets=df.sentiment_int.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=4
    )

In [8]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [19]:
BATCH_SIZE = 16

train_data_loader = create_data_loader(df_train, tokenizer, BATCH_SIZE, 160)
val_data_loader = create_data_loader(df_val, tokenizer, BATCH_SIZE, 160)
test_data_loader = create_data_loader(df_test, tokenizer, BATCH_SIZE, 160)

In [20]:
class SentimentClassifier(nn.Module):

    def __init__(self, n_classes=2):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        output = self.drop(pooled_output)
        return self.out(output)

In [ ]:
model = SentimentClassifier()

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

EPOCHS = 5

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.NLLLoss()

In [13]:
def train_epoch(
      model,
      data_loader,
      loss_fn,
      optimizer,
      device,
      scheduler,
      n_examples
    ):
    
    model = model.train()
    losses = []
    correct_predictions = 0
    for d in data_loader:   
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        outputs = model(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)
    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    
    return correct_predictions.double() / n_examples, np.mean(losses)

In [14]:
def eval_model(model, data_loader, loss_fn, device, n_examples):

    model = model.eval()

    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            _, preds = torch.max(outputs, dim=1)

            loss = loss_fn(outputs, targets)

            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)

In [18]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):


    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(df_train)
    )

    print(f'Train loss {train_loss}')

    val_acc, val_loss = eval_model(
        model,
        val_data_loader,
        loss_fn,
        device,
        len(df_val)
    )

    print(f'Val   loss {val_loss} accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), 'best_model_state.bin')
        best_accuracy = val_acc

Epoch 1/5
----------


/home/ivan/PycharmProjects/pythonProject/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/ivan/PycharmProjects/pythonProject/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `ma

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/ivan/PycharmProjects/pythonProject/venv/lib/python3.10/site-packages/IPython/core/magics/e │
│ xecution.py:1319 in time                                                                         │
│                                                                                                  │
│   1316 │   │   else:                                                                             │
│   1317 │   │   │   st = clock2()                                                                 │
│   1318 │   │   │   try:                                                                          │
│ ❱ 1319 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1320 │   │   │   │   out=None                                                                  │
│   1321 │   │   │   │   # multi-line %%time case                                                  │
│   1322 │   │   │   │   if expr_val is not None:                                                  │
│ in <module>                                                                                      │
│                                                                                                  │
│ in train_epoch                                                                                   │
│                                                                                                  │
│   11 │   model = model.train()                                                                   │
│   12 │   losses = []                                                                             │
│   13 │   correct_predictions = 0                                                                 │
│ ❱ 14 │   for d in data_loader:                                                                   │
│   15 │   │   input_ids = d["input_ids"].to(device)                                               │
│   16 │   │   attention_mask = d["attention_mask"].to(device)                                     │
│   17 │   │   targets = d["targets"].to(device)                                                   │
│                                                                                                  │
│ /home/ivan/PycharmProjects/pythonProject/venv/lib/python3.10/site-packages/torch/utils/data/data │
│ loader.py:633 in __next__                                                                        │
│                                                                                                  │
│    630 │   │   │   if self._sampler_iter is None:                                                │
│    631 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    632 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  633 │   │   │   data = self._next_data()                                                      │
│    634 │   │   │   self._num_yielded += 1                                                        │
│    635 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    636 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /home/ivan/PycharmProjects/pythonProject/venv/lib/python3.10/site-packages/torch/utils/data/data │
│ loader.py:1345 in _next_data                                                                     │
│                                                                                                  │
│   1342 │   │   │   │   self._task_info[idx] += (data,)                                           │
│   1343 │   │   │   else:                                                                         │
│   1344 │   │   │   │   del self._task_info[idx]            